# Resonator Sprectoscopy

In [41]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import Zurich

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = Zurich('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

2023.01.18 19:22:10.408 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is /home/admin/Juan/qibolab/examples/laboneq_output/log
2023.01.18 19:22:10.409 laboneq.controller.controller  INFO   VERSION: laboneq 1.4.0
2023.01.18 19:22:10.410 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2023.01.18 19:22:10.524 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2023.01.18 19:22:12.399 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2023.01.18 19:22:12.545 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [42]:
from qibocal.calibrations.characterization.utils import variable_resolution_scanrange
from qibocal.data import DataUnits
from qibolab.pulses import Pulse, PulseSequence
import laboneq.simple as lo
import numpy as np


def resonator_spectroscopy(
    Device,
    qubit: int,
    start,
    stop,
    step,
    software_averages,
):
    
    sequence = PulseSequence() 
    ro_pulse = Device.create_qubit_readout_pulse(qubit, start=0)
    # ro_pulse = Device.create_qubit_readout_pulse2(7.82e9 ,qubit, start=0)
    sequence.add(ro_pulse)

    # Base_if = Device.instruments["shfqc_qa"]["settings"]["if_frequency"]
    Base_lo = Device.instruments["shfqc_qa"]["settings"]["lo_frequency"]

    # frequency_range = (
    #     variable_resolution_scanrange(
    #         lowres_width, lowres_step, highres_width, highres_step
    #     )
    # )
    
    frequency_range = np.arange(start, stop, step)
    
    fast_sweep_data = DataUnits(
        name=f"sweep_q{qubit}", quantities={"frequency": "Hz"}
    )
    
    for _ in range(software_averages):
        for freq in frequency_range:

            Device.reload_settings()
            # Device.instruments["shfqc_qa"]["settings"]["lo_frequency"] = freq + Base_lo
            Device.instruments["shfqc_qa"]["settings"]["if_frequency_0"] = freq
            # Device.native_gates["single_qubit"][qubit]["MZ"]["frequency"] = freq
            Device.apply_settings()
            
            # msr, phase, i, q = Device.execute_pulse_sequence(sequence)
            msr, phase, i, q = Device.execute_pulse_sequence(sequence)
            print(msr, phase, i , q)
            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "frequency[Hz]": freq + Base_lo,
            }
            
            # print(Device.calib[f"/logical_signal_groups/q{0}/measure_line"])
            
            fast_sweep_data.add(results)

    # Plot simulated output signals
    # if use_emulation == True:
    #     Device.plot_output_signals()
    # else:
    #     lo.show_pulse_sheet("Pulse_sheet",Device.exp)
            
    return fast_sweep_data

In [43]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def frequency_msr_phase__fast_precision(data):
        
    data_fast = data

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("MSR", "uV"),
            name="Fast",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("phase", "rad"),
            name="Fast",
        ),
        row=1,
        col=2,
    )
    
    # if len(data_fast) > 0 and len(data_fit) > 0:
    #     freqrange = np.linspace(
    #         min(data_fast.get_values("frequency", "GHz")),
    #         max(data_fast.get_values("frequency", "GHz")),
    #         2 * len(data_fast),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=freqrange,
    #             y=lorenzian(
    #                 freqrange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"The estimated {params[0]} is {data_fit.df[params[0]][0]:.1f} Hz.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"The estimated {params[1]} is {data_fit.df[params[1]][0]:.3f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Frequency (GHz)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Frequency (GHz)",
        yaxis2_title="Phase (rad)",
    )
    return fig

In [48]:
data = resonator_spectroscopy(
    Device,
    qubit = 0,
    start = -290e6,
    stop = -260e6,
    step = 2e6,
    software_averages=1,
    )

frequency_msr_phase__fast_precision(data)

2023.01.18 19:26:47.033 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.18 19:26:47.056 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 13
2023.01.18 19:26:47.056 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 2000
2023.01.18 19:26:47.057 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.18 19:26:49.200 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.18 19:26:49.777 laboneq.controller.controller  INFO   Finished near-time execution.
3.4505845120343597e-06 -0.3390580505513369 3.254137129344656e-06 -1.1476606720245854e-06
2023.01.18 19:26:51.040 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.18 19:26:51.062 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 13
2023.01.18 19:26:51.062 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 2000
2023.01.18 19:26:51.063 labon

In [47]:
Device.calib

Calibration(calibration_items={'/logical_signal_groups/q0/measure_line': SignalCalibration(amplitude=None, delay_signal=None, local_oscillator=Oscillator(uid='osc_8296', carrier_type=CarrierType.RF, frequency=5500000000, modulation_type=ModulationType.AUTO), mixer_calibration=None, precompensation=None, oscillator=Oscillator(uid='osc_8275', carrier_type=CarrierType.RF, frequency=-262000000.0, modulation_type=ModulationType.SOFTWARE), port_delay=None, port_mode=None, range=0, threshold=None), '/logical_signal_groups/q0/acquire_line': SignalCalibration(amplitude=None, delay_signal=None, local_oscillator=Oscillator(uid='osc_8297', carrier_type=CarrierType.RF, frequency=5500000000, modulation_type=ModulationType.AUTO), mixer_calibration=None, precompensation=None, oscillator=None, port_delay=1e-08, port_mode=None, range=10, threshold=None), '/logical_signal_groups/q0/flux_line': SignalCalibration(amplitude=None, delay_signal=0, local_oscillator=None, mixer_calibration=None, precompensation

In [30]:
lo.show_pulse_sheet("Pulse_sheet",Device.exp)

2023.01.18 17:46:17.762 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to /home/admin/Juan/qibolab/examples/Pulse_sheet_2023-01-18-17-46-17.html


/home/admin/Juan/qibolab/examples/Pulse_sheet_2023-01-18-17-46-17.html

Sequences

In [5]:
def resonator_spectroscopy_sequence(
    Device,
    qubit: int,
    start,
    stop,
    points,
):
    
    sequence = PulseSequence() 
    ro_pulse = Device.create_qubit_readout_pulse(qubit, start=0)
    sequence.add(ro_pulse)
    
    freqs = np.linspace(start, stop, points)
    
    return sequence, freqs

In [6]:
Device.reload_settings()

In [39]:
start = -50e6
stop = -10e6
points = 100

sequence, freqs = resonator_spectroscopy_sequence(
    Device,
    qubit = 0,
    start = start,
    stop = stop,
    points = points,
)

qubit = 0    
Base_lo = Device.instruments["shfqc_qa"]["settings"]["lo_frequency"]
    
Data = DataUnits(
    name=f"sweep_q{qubit}", quantities={"frequency": "Hz"}
)

msr, phase, i, q = Device.execute_pulse_sequence_freq(sequence, start, stop, points)

results = {
    "MSR[V]": msr[0],
    "i[V]": i[0],
    "q[V]": q[0],
    "phase[rad]": phase[0],
    "frequency[Hz]": freqs + Base_lo,
}

Data.set(results)

frequency_msr_phase__fast_precision(Data)

2023.01.11 13:46:41.714 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.11 13:46:41.913 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 35
2023.01.11 13:46:41.914 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 2000
2023.01.11 13:46:41.914 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.11 13:46:43.043 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.11 13:46:43.323 laboneq.controller.controller  INFO   Estimated RT execution time: 5.10 s.
2023.01.11 13:46:48.596 laboneq.controller.controller  INFO   Finished near-time execution.


/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



Check RS

In [40]:
# resonator_frequency = Data.get_values("frequency", "hertz")[np.argmax(Data.df["MSR"])]
resonator_frequency = Data.get_values("frequency", "hertz")[np.argmin(Data.df["MSR"])]
freq = resonator_frequency -  Device.instruments["shfqc_qa"]["settings"]["lo_frequency"]
freq

-16060606.060606003

In [41]:
Device.reload_settings()
Device.instruments["shfqc_qa"]["settings"]["if_frequency"] = freq
Device.apply_settings()

In [42]:
sequence = PulseSequence() 
ro_pulse = Device.create_qubit_readout_pulse(0, start=0)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

2023.01.11 13:47:25.727 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.11 13:47:25.749 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 13
2023.01.11 13:47:25.750 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 2000
2023.01.11 13:47:25.750 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.11 13:47:26.843 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.11 13:47:27.463 laboneq.controller.controller  INFO   Finished near-time execution.
7.643473055476006e-06 -3.046977098201759 -7.609285990477132e-06 -7.221129170123958e-07


--------------------------------------------------

# Qubit Spectroscopy

In [43]:
from qibolab.paths import qibolab_folder
from qibolab.instruments.zhinst import SHFQC_QA

import numpy as np
from qibolab.pulses import Pulse, PulseSequence, ReadoutPulse, PulseShape
from qibocal.data import DataUnits
import laboneq.simple as lo

# runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
runcard = qibolab_folder / "runcards" / "zhinst.yml"
use_emulation = False
Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation)

2023.01.11 13:47:35.160 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is /home/admin/Juan/qibolab/examples/laboneq_output/log
2023.01.11 13:47:35.161 laboneq.controller.controller  INFO   VERSION: laboneq 1.4.0
2023.01.11 13:47:35.161 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2023.01.11 13:47:35.351 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2023.01.11 13:47:36.286 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2023.01.11 13:47:36.428 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [44]:
sequence = PulseSequence() 
ro_pulse = Device.create_qubit_readout_pulse(0, start=0)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

2023.01.11 13:47:36.944 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.11 13:47:36.966 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 13
2023.01.11 13:47:36.967 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 2000
2023.01.11 13:47:36.967 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.11 13:47:38.343 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.11 13:47:38.936 laboneq.controller.controller  INFO   Finished near-time execution.
8.967973526160987e-06 -3.0689179014901673 -8.944301235574357e-06 -6.511716925875816e-07


In [45]:
def qubit_spectroscopy(
    Device,
    qubit: int,
    start,
    end,
    step,
    software_averages,
):

    # Device.reload_settings()

    sequence = PulseSequence()
    qd_pulse = Device.create_qubit_drive_pulse(qubit, start=0, duration=1000)
    ro_pulse = Device.create_qubit_readout_pulse(qubit, start=1000)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    center_frequency = Device.instruments["shfqc_qc"]["settings"]["lo_frequency"]

    freqrange = np.arange(start, end, step)

    # platform.ro_port[qubit].lo_frequency = (
    #     platform.characterization["single_qubit"][qubit]["resonator_freq"]
    #     - ro_pulse.frequency
    # )

    data = DataUnits(name=f"sweep_q{qubit}", quantities={"frequency": "Hz"})
    
    for _ in range(software_averages):
        for freq in freqrange:
            
            
            Device.reload_settings()
            
            # Device.instruments["shfqc_qa"]["settings"]["if_frequency"] = 0
            # Device.native_gates["single_qubit"][0]["MZ"]["frequency"] = 0
            
            Device.instruments["shfqc_qc"]["settings"]["if_frequency"] = freq
            Device.apply_settings()
            
            # platform.qd_port[qubit].lo_frequency = freq - qd_pulse.frequency
            # msr, phase, i, q = platform.execute_pulse_sequence(sequence)[
            #     ro_pulse.serial
            # ]
            
            # msr, phase, i, q = Device.execute_pulse_sequence(sequence)
            msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
            print(msr, phase, i , q)
            
            results = {
                "MSR[V]": msr,
                "i[V]": i,
                "q[V]": q,
                "phase[rad]": phase,
                "frequency[Hz]": freq + center_frequency ,
            }
            
            data.add(results)

    # Plot output signals
    if use_emulation == True:
        Device.plot_output_signals()
    else:
        lo.show_pulse_sheet("Pulse_sheet",Device.exp)

    if Device.resonator_type == "3D":
        qubit_frequency = data.get_values("frequency", "Hz")[
            np.argmin(data.get_values("MSR", "V"))
        ]
    else:
        qubit_frequency = data.get_values("frequency", "Hz")[
            np.argmax(data.get_values("MSR", "V"))
        ]

    return data


In [46]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def frequency_msr_phase__fast_precision(data):
        
    data_fast = data

    # try:
    #     data_fit = Data.load_data(folder, routine, format, f"fit_q{qubit}")
    # except:
    #     data_fit = Data(
    #         quantities=[
    #             "popt0",
    #             "popt1",
    #             "popt2",
    #             "popt3",
    #             "label1",
    #             "label2",
    #         ]
    #     )

    fig = make_subplots(
        rows=1,
        cols=2,
        horizontal_spacing=0.1,
        vertical_spacing=0.1,
        subplot_titles=(
            "MSR (V)",
            "phase (rad)",
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("MSR", "uV"),
            name="Fast",
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Scatter(
            x=data_fast.get_values("frequency", "GHz"),
            y=data_fast.get_values("phase", "rad"),
            name="Fast",
        ),
        row=1,
        col=2,
    )
    
    # if len(data_fast) > 0 and len(data_fit) > 0:
    #     freqrange = np.linspace(
    #         min(data_fast.get_values("frequency", "GHz")),
    #         max(data_fast.get_values("frequency", "GHz")),
    #         2 * len(data_fast),
    #     )
    #     params = [i for i in list(data_fit.df.keys()) if "popt" not in i]
    #     fig.add_trace(
    #         go.Scatter(
    #             x=freqrange,
    #             y=lorenzian(
    #                 freqrange,
    #                 data_fit.get_values("popt0"),
    #                 data_fit.get_values("popt1"),
    #                 data_fit.get_values("popt2"),
    #                 data_fit.get_values("popt3"),
    #             ),
    #             name="Fit",
    #             line=go.scatter.Line(dash="dot"),
    #         ),
    #         row=1,
    #         col=1,
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.25,
    #             showarrow=False,
    #             text=f"The estimated {params[0]} is {data_fit.df[params[0]][0]:.1f} Hz.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    #     fig.add_annotation(
    #         dict(
    #             font=dict(color="black", size=12),
    #             x=0,
    #             y=-0.30,
    #             showarrow=False,
    #             text=f"The estimated {params[1]} is {data_fit.df[params[1]][0]:.3f} uV.",
    #             textangle=0,
    #             xanchor="left",
    #             xref="paper",
    #             yref="paper",
    #         )
    #     )
    fig.update_layout(
        showlegend=True,
        uirevision="0",  # ``uirevision`` allows zooming while live plotting
        xaxis_title="Frequency (GHz)",
        yaxis_title="MSR (uV)",
        xaxis2_title="Frequency (GHz)",
        yaxis2_title="Phase (rad)",
    )
    return fig

Pulse by pulse

In [47]:
# # runcard = "/home/admin/Juan/qibolab/src/qibolab/runcards/zhinst.yml"
# runcard = qibolab_folder / "runcards" / "zhinst.yml"

# use_emulation = False
# data = qubit_spectroscopy(
#     Device = SHFQC_QA('EL_ZURO', "DEV12146", runcard, use_emulation=use_emulation),
#     qubit = 0,
#     start = -25_000_000,
#     end = -15_000_000,
#     step = 250_000,
#     software_averages=1,
#     )

# frequency_msr_phase__fast_precision(data)

Single sequence

In [56]:
def qubit_spectroscopy_sequence(
    Device,
    qubit: int,
):

    sequence = PulseSequence()
    qd_pulse = Device.create_qubit_drive_pulse(qubit, start=0, duration=1000)
    ro_pulse = Device.create_qubit_readout_pulse(qubit, start=1000)
    sequence.add(qd_pulse)
    sequence.add(ro_pulse)

    return sequence

In [57]:
Device.reload_settings()
Device.apply_settings()

In [58]:
start = -50e6
stop = 40e6
points = 100

sequence = qubit_spectroscopy_sequence(
    Device,
    qubit = 0,
)

qubit = 0    
Base_lo = Device.instruments["shfqc_qc"]["settings"]["lo_frequency"]
    
Data = DataUnits(
    name=f"sweep_q{qubit}", quantities={"frequency": "Hz"}
)

freqs = np.linspace(start, stop, points)

msr, phase, i, q = Device.execute_pulse_sequence_freq(sequence, start, stop, points)

results = {
    "MSR[V]": msr[0],
    "i[V]": i[0],
    "q[V]": q[0],
    "phase[rad]": phase[0],
    "frequency[Hz]": freqs + Base_lo,
}

Data.set(results)

frequency_msr_phase__fast_precision(Data)

2023.01.11 13:50:53.458 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.01.11 13:50:53.781 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 64
2023.01.11 13:50:53.782 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 6000
2023.01.11 13:50:53.782 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.01.11 13:50:54.783 laboneq.controller.controller  INFO   Starting near-time execution...
2023.01.11 13:50:55.138 laboneq.controller.controller  INFO   Estimated RT execution time: 5.20 s.
2023.01.11 13:51:00.523 laboneq.controller.controller  INFO   Finished near-time execution.


/opt/anaconda3/envs/JQibo/lib/python3.9/site-packages/pandas/core/construction.py:617: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



In [9]:
qubit_frequency = Data.get_values("frequency", "hertz")[np.argmin(Data.df["MSR"])]
freq = qubit_frequency -  Device.instruments["shfqc_qc"]["settings"]["lo_frequency"]
freq

-20707070.70707035

Check QS

In [28]:
Device.reload_settings()
# Device.instruments["shfqc_qc"]["settings"]["if_frequency"] = freq
Device.apply_settings()

In [29]:
sequence = PulseSequence()
qd_pulse = Device.create_qubit_drive_pulse(0, start=0, duration=1000) 
ro_pulse = Device.create_qubit_readout_pulse(0, start=1000)
sequence.add(qd_pulse)
sequence.add(ro_pulse)
    
msr, phase, i, q = Device.execute_pulse_sequence_NoSamples(sequence)
print(msr, phase, i , q)

# msr, phase, i, q = Device.execute_pulse_sequence(sequence)
# print(msr, phase, i , q)

2022.12.09 13:10:06.449 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 13:10:06.475 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 40
2022.12.09 13:10:06.476 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 6000
2022.12.09 13:10:06.476 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2022.12.09 13:10:07.209 laboneq.controller.controller  INFO   Starting near-time execution...
2022.12.09 13:10:08.236 laboneq.controller.controller  INFO   Finished near-time execution.
4.457683753667133e-05 -2.230818304052491 -2.733158423175067e-05 -3.521475469392777e-05
2022.12.09 13:10:08.584 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2022.12.09 13:10:08.609 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 39
2022.12.09 13:10:08.609 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 6000
2022.12.09 13:10:08.609 laboneq